In [ ]:
# import rospy
# import tf2_ros
# import tf
# import numpy as np
# import tf.transformations as tf_trans
# from geometry_msgs.msg import Pose, Point, Quaternion, PoseStamped
# import sys
# import moveit_commander
# import moveit_msgs.msg
# from sensor_msgs.msg import JointState
# from ros_numpy import numpify, msgify

# rospy.init_node('cartesian_test_client')

# moveit_commander.roscpp_initialize(sys.argv)
# move_group = moveit_commander.MoveGroupCommander("arm") 
# scene = moveit_commander.PlanningSceneInterface()
# robot = moveit_commander.RobotCommander()

# display_trajectory_publisher = rospy.Publisher(
#             "/move_group/display_planned_path",
#             moveit_msgs.msg.DisplayTrajectory,
#             queue_size=10,
#         )

# grasp_pose = Pose()
# grasp_pose.position.x = 0.6
# grasp_pose.position.y = 0
# grasp_pose.position.z = 0.6
# grasp_pose.orientation.x = 0
# grasp_pose.orientation.y = 0.707
# grasp_pose.orientation.z = 0
# grasp_pose.orientation.w = 0.707

# grasp_pose_mat = numpify(grasp_pose)
# pre_grasp_shift = np.eye(4)
# pre_grasp_shift[0,3] = 0.0
# pre_grasp_pose_mat = np.dot(grasp_pose_mat, pre_grasp_shift)
# pre_grasp_pose = msgify(Pose, pre_grasp_pose_mat)

# move_group.set_start_state_to_current_state()
# trans = tf.transformations.translation_from_matrix(pre_grasp_pose_mat).tolist()
# quat = tf.transformations.quaternion_from_matrix(pre_grasp_pose_mat).tolist()
# move_group.clear_pose_targets()
# move_group.set_pose_target(trans + quat)
# plan_result = move_group.plan()

# moveit_robot_state = robot.get_current_state()
# start_position_list = list(moveit_robot_state.joint_state.position)
# print plan_result[1].joint_trajectory.points[-1].positions
# for joint_name, joint_value in zip(plan_result[1].joint_trajectory.joint_names, 
#                                    plan_result[1].joint_trajectory.points[-1].positions):
#     start_position_list[moveit_robot_state.joint_state.name.index(joint_name)] = joint_value
# moveit_robot_state.joint_state.position = tuple(start_position_list)
# # print moveit_robot_state

# move_group.set_start_state(moveit_robot_state)
# move_group.clear_pose_targets()
# (approach_plan, fraction) = move_group.compute_cartesian_path([grasp_pose], 0.001, 0, False)
# print 'fraction ', fraction

# display_trajectory = moveit_msgs.msg.DisplayTrajectory()
# display_trajectory.trajectory_start = move_group.get_current_state()
# display_trajectory.trajectory.append(plan_result[1])
# display_trajectory.trajectory.append(approach_plan)
# # Publish
# display_trajectory_publisher.publish(display_trajectory)

In [2]:
import rospy
import tf2_ros
import tf
import numpy as np
import tf.transformations as tf_trans
from geometry_msgs.msg import Pose, Point, Quaternion, PoseStamped
import sys
import moveit_commander
import moveit_msgs.msg
from moveit_msgs.srv import GetPositionFK, GetPositionFKRequest
from sensor_msgs.msg import JointState
from ros_numpy import numpify, msgify

rospy.init_node('cartesian_test_client')

moveit_commander.roscpp_initialize(sys.argv)
move_group = moveit_commander.MoveGroupCommander("arm") 
scene = moveit_commander.PlanningSceneInterface()
robot = moveit_commander.RobotCommander()

display_trajectory_publisher = rospy.Publisher(
            "/move_group/display_planned_path",
            moveit_msgs.msg.DisplayTrajectory,
            queue_size=10,
        )

fk_srv = rospy.ServiceProxy("/compute_fk", GetPositionFK)
fk_srv.wait_for_service()

move_group.clear_pose_targets()
move_group.set_position_target([0.7,0,0.8])
plan_result = move_group.plan()

moveit_robot_state = robot.get_current_state()
start_position_list = list(moveit_robot_state.joint_state.position)
print plan_result[1].joint_trajectory.points[-1].positions
for joint_name, joint_value in zip(plan_result[1].joint_trajectory.joint_names, 
                                   plan_result[1].joint_trajectory.points[-1].positions):
    start_position_list[moveit_robot_state.joint_state.name.index(joint_name)] = joint_value
moveit_robot_state.joint_state.position = tuple(start_position_list)
# print moveit_robot_state

fk_request = GetPositionFKRequest()
fk_request.robot_state = moveit_robot_state
fk_request.fk_link_names = ['wrist_roll_link']
fk_request.header.frame_id = 'base_link'
fk_resp = fk_srv.call(fk_request)
next_gripper_pose = fk_resp.pose_stamped[0].pose

# calculate the pose to move forward
next_gripper_pose_mat = numpify(next_gripper_pose)
grasp_shift = np.eye(4)
grasp_shift[0,3] = 0.1
grasp_pose_mat = np.dot(next_gripper_pose_mat, grasp_shift)
grasp_pose = msgify(Pose, grasp_pose_mat)

move_group.set_start_state(moveit_robot_state)
move_group.clear_pose_targets()
(approach_plan, fraction) = move_group.compute_cartesian_path([grasp_pose], 0.001, 0, False)
print 'fraction ', fraction

display_trajectory = moveit_msgs.msg.DisplayTrajectory()
display_trajectory.trajectory_start = move_group.get_current_state()
display_trajectory.trajectory.append(plan_result[1])
display_trajectory.trajectory.append(approach_plan)
# Publish
display_trajectory_publisher.publish(display_trajectory)

(0.3232642040754246, -0.35012975381601125, -0.5288743193101653, 0.7744717685852098, -0.888779954498315, 1.9376827064020554, -0.15794172936738327)
fraction  1.0
